<a href="https://colab.research.google.com/github/YashShekhar/Generative-AI/blob/main/1_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langchain
Model, Promt, Prompt Templates and Chain

In [1]:
!pip install -qU "langchain[groq]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 6.5 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
apiKey = userdata.get('GROQ_API_KEY')

In [3]:
from langchain.chat_models import init_chat_model
model = init_chat_model(
    model="llama-3.3-70b-versatile",
    model_provider="groq",
    api_key=apiKey
)

In [4]:
response = model.invoke("Suggest me a title of movie in hindi to watch")
print(response.content)

Here are a few suggestions for Hindi movies to watch:

1. "3 Idiots" (2009) - A comedy-drama film that explores the lives of three friends and their experiences in engineering college.
2. "Dilwale Dulhania Le Jayenge" (1995) - A romantic drama film that tells the story of two young lovers who fall in love on a trip to Europe.
3. "Lagaan" (2001) - A historical sports drama film that tells the story of a group of villagers who challenge the British colonial authorities to a game of cricket.
4. "Kuch Kuch Hota Hai" (1998) - A romantic comedy-drama film that explores the relationships between three friends and their experiences with love.
5. "Andhadhun" (2018) - A black comedy thriller film that tells the story of a blind pianist who becomes embroiled in a murder mystery.

Which one of these movies sounds interesting to you?

Alternatively, you can also consider some newer releases like:

* "Gully Boy" (2019)
* "Badhaai Ho" (2018)
* "Stree" (2018)
* "Article 15" (2019)
* "Jabariya Jodi" (2

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English to Italian"),
    HumanMessage("hi")
]

response = model.invoke(messages)
print(response.content)

Ciao!


In [6]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from english to {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("user", "{text}")
    ]
)

In [7]:
prompt = prompt_template.invoke(
    {
        "language": "hindi",
        "text": "what is you age"
    }
)

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from english to hindi', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is you age', additional_kwargs={}, response_metadata={})])

In [8]:
model.invoke(prompt)

AIMessage(content='आपकी उम्र क्या है?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 46, 'total_tokens': 57, 'completion_time': 0.04, 'prompt_time': 0.002310338, 'queue_time': 0.242108346, 'total_time': 0.042310338}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_90c1d253ff', 'finish_reason': 'stop', 'logprobs': None}, id='run-c9d42fae-65bb-4b39-b8bc-7918797f4db9-0', usage_metadata={'input_tokens': 46, 'output_tokens': 11, 'total_tokens': 57})

In [9]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")

prompt = prompt_template.invoke({"topic": "cats"})

response = model.invoke(prompt)

response.content

'Why did the cat join a band? Because it wanted to be the purr-cussionist.'

In [10]:
movie_title_template = PromptTemplate.from_template("Suggest me a title of language {language} and of genre {genre} make sure to give only one suggestion")

movie_title_prompt = movie_title_template.invoke({
    "language": "hindi",
    "genre": "comedy"
})

response = model.invoke(movie_title_prompt)

print(response.content)

"Golmaal" - This title is in Hindi and falls under the comedy genre, it's also a popular Bollywood movie franchise.


In [11]:
from langchain_core.output_parsers import StrOutputParser

movie_title_chain = movie_title_template | model | StrOutputParser()

response = movie_title_chain.invoke({"language": "english", "genre":"horror"})

print(response)

"The Shining"


In [12]:
movie_summary_prompt = PromptTemplate.from_template("Give me 2-3 line summary of the movie {movie_title}")

In [13]:
from langchain_core.runnables import RunnableLambda
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

print_title_step = RunnableLambda(lambda x : print(x["movie_title"]))

composed_chain = {"movie_title": movie_title_chain} | print_title_step | movie_summary_prompt | model | StrOutputParser()

response = composed_chain.invoke({"language": "english", "genre":"horror"})

response

"The Shining"


'There is no information available for a movie titled "None." If you could provide the actual title of the movie, I\'d be happy to give you a summary. Please let me know the correct title, and I\'ll do my best to assist you.'